# Healpy Exercises

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import h5py
import os
from astropy.coordinates import SkyCoord
import healpy as hp
import numpy as np

## Making a map out of a large HDF5 Catalogue

In [ ]:
datadir = os.path.expanduser("~/DATA")

In [ ]:
mcalf = "DESY3_metacal_v03-004.h5"
goldf = "DESY3_GOLD_2_2.1.h5"

In [ ]:
mcal=h5py.File(datadir+ "/" +mcalf)
gold=h5py.File(datadir+ "/" +goldf)

In [ ]:
def cat2hpx(lon, lat, nside, radec=True):
    """
    Convert a catalogue to a HEALPix map of number counts per resolution
    element.

    Parameters
    ----------
    lon, lat : (ndarray, ndarray)
        Coordinates of the sources in degree. If radec=True, assume input is in the icrs
        coordinate system. Otherwise assume input is glon, glat

    nside : int
        HEALPix nside of the target map

    radec : bool
        Switch between R.A./Dec and glon/glat as input coordinate system.

    Return
    ------
    hpx_map : ndarray
        HEALPix map of the catalogue number counts in Galactic coordinates

    """

    npix = hp.nside2npix(nside)

    if radec:
        eq = SkyCoord(lon, lat, unit='deg')
        l, b = eq.galactic.l.value, eq.galactic.b.value
    else:
        l, b = lon, lat

    # conver to theta, phi
    theta = np.radians(90. - b)
    phi = np.radians(l)

    # convert to HEALPix indices
    indices = hp.ang2pix(nside, theta, phi)

    idx, counts = np.unique(indices, return_counts=True)

    # fill the fullsky map
    hpx_map = np.zeros(npix, dtype=int)
    hpx_map[idx] = counts

    return hpx_map

In [ ]:
indexes=np.arange(gold['catalog']['gold']['ra'].len())
idx_chunks = np.array_split(indexes,10000)

In [ ]:
import gc
import tqdm
nside=256
npix = hp.nside2npix(nside)
fullmap = np.zeros(npix)
for idx_chunk in tqdm.tqdm(idx_chunks):
  chunk = gold['catalog']['gold']['ra'][idx_chunk], gold['catalog']['gold']['dec'][idx_chunk]
  fullmap+=cat2hpx(chunk[0],chunk[1],nside,True)
  del chunk
  gc.collect() 

In [ ]:
hp.mollview(fullmap)